In [2]:
!pip install plinkio
##Load data:
import os
import re
import numpy as np
import pandas as pd
from plinkio import plinkfile
import time
#from scipy.linalg.blas import dsyrk 
    #--can't find a way to get this working. Perhaps blas routines are missing.
    
data_path = '/home/jovyan/work/LEAP/leap/regression/dataset1'
os.chdir(data_path)    

  Running setup.py bdist_wheel for plinkio ... - \ | / - \ | / - \ done
  Stored in directory: /home/jovyan/.cache/pip/wheels/6f/e2/58/0f2d910f7aa9154227bf313074794f255534879d9f0b66e863
Successfully built plinkio


In [3]:
"""
author: gene burinskiy

Goal: 
Finding a set of individuals who are related to other individuals in the study. 
LEAP employs a greedy algorithm to find a small subset of such individuals, 
such that after their exclusion, there are no related individuals in the study. 
These individuals are excluded from the analysis in stages 3 and 4 below, 
but after fitting a model in stage 4, their liabilities are estimated along with 
other indviduals. All individuals are considered in the GWAS stage (stage 5).

source: 
https://github.com/omerwe/LEAP/blob/master/leap/regression/Leap_example.ipynb
"""

'\nauthor: gene burinskiy\n\nGoal: \nFinding a set of individuals who are related to other individuals in the study. \nLEAP employs a greedy algorithm to find a small subset of such individuals, \nsuch that after their exclusion, there are no related individuals in the study. \nThese individuals are excluded from the analysis in stages 3 and 4 below, \nbut after fitting a model in stage 4, their liabilities are estimated along with \nother indviduals. All individuals are considered in the GWAS stage (stage 5).\n\nsource: \nhttps://github.com/omerwe/LEAP/blob/master/leap/regression/Leap_example.ipynb\n'

In [3]:
##Load data:
bed = plinkfile.open("dataset1")

loci = bed.get_loci()
print("Length of locuses", len(loci))
chromosomes = np.unique([x.chromosome for x in loci])
print("# of chromosomes in data:",chromosomes)

samples = bed.get_samples()
print("Number of individuals in data:", len(samples))

Length of locuses 10499
# of chromosomes in data: [ 1  2  3  4  5  6  7  8  9 10]
Number of individuals in data: 1000


In [5]:
##Place data into a dataframe:
mat = np.zeros((len(loci),len(samples)), dtype='int16') #1/4 of the taken up space by using int16

##don't know a faster method of extracting the data from the bed file.
i=0
for row in bed:
    mat[i,:] = np.array([snp for snp in row])
    i+=1
    
#this matrix is equivalent to transposed bed.val
print("Data type:", mat.dtype)
print("Size of bed matrix: %4.0fmb\n" %(mat.nbytes/(1024**2)))

#create a multi-indexed column space
tuples = [(x.chromosome,x.name) for x in loci]
ml_index = pd.MultiIndex.from_tuples(tuples, names = ['chromosome', 'snp'])

df = pd.DataFrame(mat.transpose(), columns=ml_index, index = [x.iid for x in bed.get_samples()]) 
df.info()
df.iloc[:5,:5]

Data type: int16
Size of bed matrix:   20mb

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, person1 to person1000
Columns: 10499 entries, (1, csnp18) to (10, snp10483)
dtypes: int16(10499)
memory usage: 20.0+ MB


chromosome      1                            
snp        csnp18 csnp35 csnp59 csnp78 csnp85
person1         2      1      1      2      1
person2         1      0      2      2      2
person3         0      2      2      2      2
person4         2      1      2      2      1
person5         0      1      2      1      2

In [50]:
##compute covariance matrix between individuals, remove those who are too close to each other.
#they LEAP code uses dsyrk which halves the computational time. Alas, we can't use it y

df = df.astype('float32')-df.astype('float32').mean() 
df.info() #roughly doubled memory usage though still not the 80mb it was earlier

cov = np.dot(df, df.transpose())/df.shape[1] #having difficulties with scipy's linalg module
#note that the above takes more than half the time of np.cov
print("\nCovariance shape:" , cov.shape)
print("Covariance memory usage in mb:", cov.nbytes/(1024**2))
cov[:5,:5]

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, person1 to person1000
Columns: 10499 entries, (1, csnp18) to (10, snp10483)
dtypes: float32(10499)
memory usage: 40.1+ MB

Covariance shape: (1000, 1000)
Covariance memory usage in mb: 3.814697265625


array([[ 0.36813205,  0.00128837, -0.00865505, -0.00119463,  0.00389233],
       [ 0.00128837,  0.35822782,  0.00339448,  0.00228265,  0.00136904],
       [-0.00865505,  0.00339448,  0.36281955,  0.00443562, -0.00057362],
       [-0.00119463,  0.00228265,  0.00443562,  0.3630724 ,  0.0018387 ],
       [ 0.00389233,  0.00136904, -0.00057362,  0.0018387 ,  0.37096035]], dtype=float32)

In [79]:
cutoff = .05
bool_arr =  np.tril(cov, k=-1)>cutoff
y_idx,_ = np.where(bool_arr)
print("shape of y:", y_idx.shape)
print("\nremoving %d individuals" %y_idx.shape[0])

#note, they marked 54 so we marked more peeps, we effectively remove 47. Something doesn't line up.
indxToKeep = set(range(cov.shape[0]))
[indxToKeep.remove(i) for i in np.unique(y_idx)]
keepArr = np.array(list(indxToKeep))
keepArr.shape

shape of y: (56,)

removing 56 individuals


(953,)

In [75]:
t_set = set(range(5))
[t_set.remove(i) for i in [2,3]]
t_set

{0, 1, 4}

In [55]:
#exploring different ways to exclude individuals found above.
cov_m = np.ma.array(cov,mask=False)
cov_m.mask[y_idx,:] = True
cov_m.mask[:,x_idx] = True

print(cov_m.sum())

cov_c = np.delete(np.delete(cov, y_idx, axis=0), x_idx, axis=1)
print(cov_c.sum())

14.6601
14.6601


In [60]:
#trying to match the authors' results but uh, to no avail.
corr = np.corrcoef(df)
bool_arr =  np.tril(corr, k=-1)>cutoff
y_idx,x_idx = np.where(bool_arr)

corr_c = np.delete(np.delete(corr, y_idx, axis=0), x_idx, axis=1)
corr_c.shape

(933, 934)

In [81]:
#with multi-index, we index by using the number of the chromosome. 
#This avoids copying of data -> we use views on the data. Immeasurably more efficient
for chrom in chromosomes:
    print("Working on chromosome: %s" %chrom)
    
    exclude_chrom = set(chromosomes)
    exclude_chrom.remove(chrom) #set all chromosomes except current
    exclude_chrom = list(exclude_chrom)
    
    t0 = time.time()
    #Note that the original code puts cov, s, U into a dictionary called "eigen"
    #They do not actually perform an SVD decomposition. Instead, they compute
    #the covariance matrix, decompose that and use an equivalence relation between
    #SVD and the decomposition of the covariance matrix. However, it seems that a 
    #set could be saved if they just performed the SVD decomposition, albeit at a higher computing cost
    cov = np.dot(df[exclude_chrom], df[exclude_chrom].transpose())/df[exclude_chrom].shape[1]
    
    s,U = np.linalg.eigh(cov, 'L') #would use scipy except -again- can't get it to load.
    
    print("Took %.2f seconds" %(time.time()-t0))
print("Note that LEAP's original code runs 2-3 times slower for this step")    

Working on chromosome: 1
Took 0.30 seconds
Working on chromosome: 2
Took 0.21 seconds
Working on chromosome: 3
Took 0.21 seconds
Working on chromosome: 4
Took 0.19 seconds
Working on chromosome: 5
Took 0.18 seconds
Working on chromosome: 6
Took 0.18 seconds
Working on chromosome: 7
Took 0.29 seconds
Working on chromosome: 8
Took 0.25 seconds
Working on chromosome: 9
Took 0.20 seconds
Working on chromosome: 10
Took 0.20 seconds
Note that LEAP's original code runs 2-3 times slower for this step


In [100]:
##Our calc_h2 function for Step 3
#uses the calc_h2.calc_h2 functions
from sklearn.linear_model import LogisticRegression
from scipy import stats

#read in phenofile:
phenos = pd.read_csv("dataset1.phe", sep=' ', header=None, engine='c')
phenos.columns = ['fam', 'person', 'pheno']
phenos.set_index(keys = 'person', inplace=True)
phenos.iloc[:5,:]

,fam,pheno
person,,
person1,FAM1,0
person2,FAM1,0
person3,FAM1,0
person4,FAM1,0
person5,FAM1,0


In [101]:
#another part of the calc_h2 function
prev, prevalence = [.001]*2

numRemovePCs=10 #their default value; as far as I'm aware, they do not input different values

if numRemovePCs>0:
    t_cov = cov -  (U[:,-numRemovePCs:]*s[-numRemovePCs:]).dot(U[:,-numRemovePCs:].transpose())
        
pheUnique = phenos.pheno.unique()
isCaseControl = pheUnique.shape[0] == 2 #trivial condition for us

if ~np.all(pheUnique == np.array([0,1])):
    pheMean = phenos.pheno.mean()
    phenos.pheno[phenos.pheno <= pheMean] = 0
    phenos.pheno[phenoes.pheno > pheMean] = 1
    
#probs, thresholds = calcLiabThreholds(U, S, keepArr, phe, numRemovePCs, prevalence, covar) 

#This is equivalent to an SVD decomposition; note their covar parameter is defaulted to None
G = U[:, -numRemovePCs:] * np.sqrt(s[-numRemovePCs:])

#perform a regularized logistic regression. I trust their parameter settings.
Logreg = LogisticRegression(penalty='l2', C=500000, fit_intercept=True)
Logreg.fit(G[keepArr, :], phenos.pheno.iloc[keepArr])

#Compute individual thresholds
probs = Logreg.predict_proba(G)[:,1]

#Compute thresholds
P = np.sum(phenos.pheno==1) / float(phenos.pheno.shape[0])
#K = prev --why, why in the (insert explicative) hell do they do this?
Ki = prev*(1-prev) / (P*(1-prev)) * probs / (1 + prev*(1-prev) / (P*(1-prev))*probs - probs)
thresholds = stats.norm(0,1).isf(Ki)
thresholds[Ki>=1.] = -999999999
thresholds[Ki<=0.] = 999999999

In [86]:
#h2 = calcH2Binary(XXT, phe, probs, thresholds, keepArr, prevalence, h2coeff)   


True

In [33]:
%%timeit
np.dot(df, df.transpose())/df.shape[1]

1 loop, best of 3: 154 ms per loop


In [32]:
%%timeit
np.cov(df)

1 loop, best of 3: 354 ms per loop
